# RTM Simulation: Ballistic 1-D Propagation

This notebook demonstrates the **ballistic transport regime** in the RTM (Relatividad Temporal Multiescala) framework.

## Theoretical Background

In ballistic transport, a particle moves at constant velocity without scattering. The time $T$ to traverse a distance $L$ scales as:

$$T \propto L^\alpha \quad \text{with} \quad \alpha = 1$$

This is the simplest case: $T = L/v$ where $v$ is constant.

**Expected result:** $\alpha \approx 1.00$

This serves as the **lower benchmark** for RTM temporal-relativity tests.

## 1. Setup and Imports

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd

# For reproducibility
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

print("Libraries loaded successfully!")

## 2. Define Simulation Parameters

In [ ]:
# System sizes (in arbitrary length units)
SYSTEM_SIZES = np.array([10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000])

# Number of trials per system size
N_TRIALS = 100

# Base velocity (constant)
BASE_VELOCITY = 1.0

# Small velocity noise (1% standard deviation)
VELOCITY_NOISE_STD = 0.01

print(f"System sizes: {SYSTEM_SIZES}")
print(f"Number of trials per size: {N_TRIALS}")
print(f"Total simulations: {len(SYSTEM_SIZES) * N_TRIALS}")

## 3. Define the Ballistic Transport Model

In pure ballistic transport:
$$T = \frac{L}{v}$$

We add small Gaussian noise to the velocity to simulate real-world conditions.

In [ ]:
def simulate_ballistic_transport(L, n_trials, base_velocity, noise_std, rng):
    """
    Simulate ballistic transport with small velocity fluctuations.
    
    Parameters:
    -----------
    L : float - System size / distance
    n_trials : int - Number of independent trials
    base_velocity : float - Mean velocity
    noise_std : float - Standard deviation of velocity noise (relative)
    rng : numpy.random.Generator
    
    Returns:
    --------
    numpy.ndarray - Array of traversal times
    """
    # Generate velocities with small Gaussian noise
    velocities = base_velocity + rng.normal(0, noise_std * base_velocity, n_trials)
    # Ensure velocities are positive
    velocities = np.maximum(velocities, base_velocity * 0.1)
    
    # Calculate traversal times: T = L / v
    times = L / velocities
    
    return times

print("Ballistic transport function defined.")

## 4. Run the Simulation

In [ ]:
# Storage for results
results = []

print("Running simulations...\n")
print(f"{'L':>8} | {'T_mean':>12} | {'T_std':>10} | {'T_sem':>10}")
print("-" * 50)

for L in SYSTEM_SIZES:
    # Run trials
    times = simulate_ballistic_transport(L, N_TRIALS, BASE_VELOCITY, 
                                         VELOCITY_NOISE_STD, rng)
    
    # Calculate statistics
    T_mean = np.mean(times)
    T_std = np.std(times, ddof=1)
    T_sem = T_std / np.sqrt(N_TRIALS)
    
    results.append({
        'L': L,
        'T_mean': T_mean,
        'T_std': T_std,
        'T_sem': T_sem
    })
    
    print(f"{L:>8.0f} | {T_mean:>12.4f} | {T_std:>10.4f} | {T_sem:>10.4f}")

# Create DataFrame
df = pd.DataFrame(results)
print("\nSimulation complete!")

## 5. Fit Power Law: $T = T_0 \cdot L^\alpha$

We perform linear regression in log-log space:
$$\log T = \alpha \cdot \log L + \log T_0$$

In [ ]:
# Log-transform the data
log_L = np.log10(df['L'].values)
log_T = np.log10(df['T_mean'].values)

# Linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(log_L, log_T)

# Extract parameters
alpha = slope
T0 = 10**intercept
R_squared = r_value**2

print("=" * 50)
print("POWER LAW FIT RESULTS")
print("=" * 50)
print(f"\nFitted exponent: α = {alpha:.4f} ± {std_err:.4f}")
print(f"Prefactor: T₀ = {T0:.4f}")
print(f"R² = {R_squared:.6f}")
print(f"p-value = {p_value:.2e}")
print(f"\nExpected (theoretical): α = 1.00")
print(f"RTM paper reported: α ≈ 1.03")

## 6. Bootstrap Confidence Intervals

In [ ]:
# Bootstrap for confidence intervals
n_bootstrap = 1000
bootstrap_rng = np.random.default_rng(RANDOM_SEED + 1)
bootstrap_slopes = []

for _ in range(n_bootstrap):
    # Resample with replacement
    indices = bootstrap_rng.choice(len(log_L), size=len(log_L), replace=True)
    boot_log_L = log_L[indices]
    boot_log_T = log_T[indices]
    boot_slope, _, _, _, _ = stats.linregress(boot_log_L, boot_log_T)
    bootstrap_slopes.append(boot_slope)

bootstrap_slopes = np.array(bootstrap_slopes)
ci_lower = np.percentile(bootstrap_slopes, 2.5)
ci_upper = np.percentile(bootstrap_slopes, 97.5)

print(f"95% Confidence Interval: [{ci_lower:.4f}, {ci_upper:.4f}]")

# Histogram of bootstrap distribution
plt.figure(figsize=(8, 4))
plt.hist(bootstrap_slopes, bins=50, density=True, alpha=0.7, color='steelblue')
plt.axvline(alpha, color='red', linestyle='-', linewidth=2, label=f'Fitted α = {alpha:.4f}')
plt.axvline(1.0, color='green', linestyle='--', linewidth=2, label='Theoretical α = 1.00')
plt.axvline(ci_lower, color='orange', linestyle=':', linewidth=1.5)
plt.axvline(ci_upper, color='orange', linestyle=':', linewidth=1.5, label='95% CI')
plt.xlabel('α', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.title('Bootstrap Distribution of α', fontsize=14)
plt.legend()
plt.tight_layout()
plt.show()

## 7. Visualization: Log-Log Plot

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Log-log plot with fit
ax1 = axes[0]
ax1.errorbar(df['L'], df['T_mean'], yerr=df['T_sem'], 
             fmt='o', markersize=8, capsize=3, label='Simulation data')

# Fit line
L_fit = np.logspace(np.log10(df['L'].min()), np.log10(df['L'].max()), 100)
T_fit = T0 * L_fit**alpha
ax1.plot(L_fit, T_fit, 'r-', linewidth=2, 
         label=f'Fit: α = {alpha:.4f} ± {std_err:.4f}')

# Theoretical line (α = 1)
T_theoretical = T0 * L_fit**1.0
ax1.plot(L_fit, T_theoretical, 'g--', linewidth=1.5, alpha=0.7,
         label='Theoretical: α = 1.00')

ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel('System Size L', fontsize=12)
ax1.set_ylabel('Traversal Time T', fontsize=12)
ax1.set_title('Ballistic 1-D: T ∝ L^α Scaling', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: Residuals
ax2 = axes[1]
T_predicted = T0 * df['L'].values**alpha
residuals = (df['T_mean'].values - T_predicted) / T_predicted * 100

ax2.bar(range(len(df)), residuals, color='steelblue', alpha=0.7)
ax2.axhline(y=0, color='r', linestyle='--', linewidth=1)
ax2.set_xticks(range(len(df)))
ax2.set_xticklabels([f'{int(L)}' for L in df['L']], rotation=45)
ax2.set_xlabel('System Size L', fontsize=12)
ax2.set_ylabel('Residual (%)', fontsize=12)
ax2.set_title('Fit Residuals', fontsize=14)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 8. Summary and Conclusions

In [ ]:
print("=" * 60)
print("SUMMARY: Ballistic 1-D Simulation")
print("=" * 60)
print(f"""
RTM Prediction for Ballistic Regime: α = 1.00
Paper Reported Value: α ≈ 1.03

This Simulation:
  • Fitted exponent: α = {alpha:.4f} ± {std_err:.4f}
  • 95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]
  • R² = {R_squared:.6f}
  
INTERPRETATION:
The ballistic regime shows T ∝ L^1, meaning time scales linearly
with distance. This is the simplest transport regime where
particles move at constant velocity without scattering.

This serves as the LOWER BENCHMARK (α_min ≈ 1) for the RTM
framework, with higher α values indicating more complex
transport mechanisms (diffusive α≈2, fractal α≈2.5, quantum α≈3.5).

STATUS: {'✓ CONFIRMED' if abs(alpha - 1.0) < 0.1 else '⚠ NEEDS REVIEW'}
""")

## 9. Save Results

In [ ]:
# Save data to CSV
df.to_csv('output/ballistic_1d_data.csv', index=False)
print("Data saved to output/ballistic_1d_data.csv")

# Display final dataframe
df